# Deploy a trained model to ACI

In [1]:
from azureml.core import Workspace, Run
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.0.1060109


## Initialize Workspace
Initialize a workspace object from persisted configuration

In [2]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found config.json in: /home/demouser/repos/AzureAILabs/Lab01-IoT-LSTM/aml_config/config.json
Workspace name: Lab01
Azure region: eastus2
Subscription id: 952a710c-8d9c-40c1-9fec-f752138cc0b3
Resource group: jkaml


## Create a scoring script

In [7]:
%%writefile score.py
import json
import numpy
import tensorflow as tf
from azureml.assets.persistence.persistence import get_model_path
from tensorflow.python.keras.models import load_model

def init():
    global model
    
    model_path = 'best_model.h5'
    # deserialize the model file back into a sklearn model
    model = load_model(model_path)

# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = numpy.array(data)
        result = model.predict(data)
    except Exception as e:
        result = str(e)
    return json.dumps({"result": result.tolist()})

Overwriting score.py


In [ ]:
import score
import json

data = [[
    [0.0],
    [0.0],
    [0.0],
    [0.0],
    [0.0],
    [0.0],
    [0.0],
    [0.0],
    [0.0],
    [0.0],
    [0.0006985],
    [0.0033175]]]
    
score.init()
print(score.run(json.dumps({'data': data})))

## Create conda dependency file

In [4]:
%%writefile lab1.yml
name: lab1
channels:
  - defaults
dependencies:
  - pip:
    - numpy
    - tensorflow
    # Required packages for AzureML execution, history, and data preparation.
    - --extra-index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E7501C02541B433786111FE8E140CAA1
    - azureml-core

Writing lab1.yml


## Deploy web service to Azure Container Instance

In [6]:
from azureml.core.webservice import AciWebservice, Webservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = ['Lab1 sample'], 
                                               description = 'This is an IoT sample')

In [ ]:
%%time
# this will take 5-10 minutes to finish
# you can also use "az container list" command to find the ACI being deployed
service = Webservice.deploy(name = 'my-aci-svc-01',
                            deployment_config = aciconfig,
                            model_paths = ['best_model.h5'],
                            runtime = 'python',
                            conda_file = 'myenv.yml',
                            execution_script = 'score.py',
                            workspace = ws)
service.wait_for_deployment(show_output = True)

## Test web service

In [ ]:
print('web service is hosted in ACI:', service.scoring_uri)

In [ ]:
# score the first row from the test set.
test_samples = json.dumps({"data": X_test[0:1, :].tolist()})
test_samples = bytes(test_samples, encoding = 'utf8')

service.run(input_data = test_samples)